# Deploy Machine Learning Job on TrueFoundry

This notebook demonstrates a demo on how you can deploy a classification model trained on customer churn dataset as well as log the job metadata on truefoundry platform.


## Prerequisites

Before we begin, make sure you have the following prerequisites in place:

### Install `truefoundry[ml]`. 
You can install it using the following command:


In [ ]:
%pip install -U "truefoundry[ml]>=0.2.0,<1.0.0"

### Login into TrueFoundry

In order to login run the cell below. Host can be found from the TrueFoundry UI as shown below like https://app.truefoundry.com

![image.png](../common/images/host.png)

In [ ]:
!tfy login --host "<Host name of TrueFoundry UI. e.g. https://company.truefoundry.cloud>"

### Create a ML Repo

We will be using the ML Repo to store the model artifacts and results of our model training. You can create an ML Repo using the following the docs at: https://docs.truefoundry.com/docs/key-concepts#creating-an-ml-repo

### Select the `Workspace` in which you want to deploy your application.

Once you run the cell below you will get a prompt to enter your workspace. Follow the docs to

a. Create a Workspace: https://docs.truefoundry.com/docs/key-concepts#creating-a-workspace  
b. Grant Editor access to ML Repo we created above: https://docs.truefoundry.com/docs/key-concepts#grant-access-of-ml-repo-to-workspace  


Or, just get the workspace FQN for existing workspace with access: https://docs.truefoundry.com/docs/key-concepts#getting-workspace-fqn


Once done, you should have a workspace FQN that you can paste in the next cell when prompted

In [ ]:
import click

In [ ]:
WORKSPACE_FQN = click.prompt(
    "Enter the Workspace FQN",
    type=str,
)
print(f"Workspace FQN set to {WORKSPACE_FQN!r}")

### Setup Code

In [ ]:
import logging

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(name)s] %(levelname)-8s %(message)s"
)

### Code Structure

Before we proceed, let's take a quick look at the structure of the code you'll be deploying:

```text
.
|_ main.py : Contains the training code
|_ requirements.txt : Dependency file
```

Let's help you understand the key elements in the main.py code that you'll be deploying:

- **Hyperparameters and argparse:**  
  Firstly, the argparse library is used to handle hyperparameters as command-line arguments. This dynamic approach allows altering hyperparameters without modifying the code itself. These command-line hyperparameters are then passed to the train_model function.
- **`train_model` Function:**  
  The train_model function is responsible for training the K-Nearest Neighbors (KNN) classifier using the provided hyperparameters. It also calculates the metrics for evaluating the model. Then it passes all of this info to `experiment_track` function
- **`track_experiment` Function:**  
  The track_experiment function logs experiment-related details into the ML Repo. Specifically:
  - It Initializes the TrueFoundry ML client.
  - Creates an ML Repo named "churn-pred."
  - Creates a run within the ML Repo to track this experiment.
  - Logs hyperparameters and metrics.
  - Logs the trained model using the log_model method, enabling deployment via Model Deployment.


In [11]:
!pygmentize main.py

from typing import Dict, Union

import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier as Classification
from truefoundry.ml import ModelFramework, get_client


def track_experiment(ml_repo: str, model, params: Dict[str, str], metrics: Dict[str, Union[int, float]]):
    # initialize the TrueFoundry ML client.
    client = get_client()
    # create a run
    run = client.create_run(ml_repo=ml_repo, run_name="churn-train-job")
    # log the hyperparameters
    run.log_params(params)
    # log the metrics
    run.log_metrics(metrics)

    # dump the model and then save it.
    joblib.dump(model, "classifier.joblib")

    # log the model
    model_version = run.log_model(
        name="churn-model",
        # Path to the folder where the model is saved locally
        model_file_or_folder="classifier.joblib",
        # specif

## Deploying Your Machine Learning Job

Now, let's move on to the deployment steps.


### Step 1: Set Up Deployment Configuration

In this step, you will define your deployment configuration using the TrueFoundry Python SDK. We will provide explanations for each parameter and guide you through the process.


#### Name

In the provided Python script, set a unique identifier for your job using the name field.


In [ ]:
name = "churn-prediction-job"

#### Image

- Choosing the Right Approach for specifying image:
  Depending on your scenario, you can choose to deploy either a pre-built Docker image or build a Docker image from your source code.
- Using Pre-Built Images
  If you already have a Docker image that you've previously built and pushed to a container registry, you can use the `Image` class.
  The `Image class` would simply reference the pre-built image URL and use it for deployment.
- Using Build for Source Code
  In cases where you don't have a pre-built image, you'll use the `Build` option to create an image from your source code.
  This scenario applies when you want to package and deploy your application from scratch.
  - Creating DockerFile with PythonBuild
    If you don't have a Dockerfile but your application is written in Python, you can use the `PythonBuild` class.
    The `PythonBuild` class will inspect your Python code and create a Dockerfile automatically based on the code's requirements.
  - Choosing DockerBuild for Dockerfile
    If you have a pre-existing Dockerfile, you can use the `DockerBuild` class.
    This allows you to directly reference the Dockerfile present in your code repository.

In this case given we did not have a prebuilt image, and no dockerfile in our source code we are using PythonBuild, which takes our code configuration from us and templatizes a Dockerfile for us.

In the Command field, enter the command to execute your training job, including placeholders for hyperparameters like {{n_neighbors}}, {{weights}}, etc.  
These are going to be the same names we specify in the Params configuration below, so keep this in mind.


In [ ]:
from truefoundry.deploy import Build, PythonBuild, LocalSource

image = Build(
    build_source=LocalSource(local_build=False),
    build_spec=PythonBuild(
        python_version="3.11",
        command="python main.py --n_neighbors {{n_neighbors}} --weights {{weights}} --ml_repo {{ml_repo}}",
        requirements_path="requirements.txt",
    ),
)

#### Params

The `Param` option empowers you to configure hyperparameters and pass them to create distinct job runs.

For each parameter, provide the following details:

- **Name:** Enter a descriptive name for the parameter.
- **Default value:** Specify the default value for the parameter.
- **Description:** Include a brief description of the parameter's purpose.
- **Param type:** Can be either string or an ML Repo

Note that the name of Param are same as what we filled in the comman's {{}} template. `python main.py --n_neighbors {{n_neighbors}} --weights {{weights} --ml_repo {{ml_repo}}`


In [ ]:
from truefoundry.deploy import Param

params = [
    Param(
        name="n_neighbors",
        default=5,
        description="Number of neighbors to use by default",
    ),
    Param(
        name="weights",
        default="uniform",
        description="Weight function used in prediction.  Possible values: uniform, distance",
    ),
     Param(
        name="ml_repo",
        param_type="ml_repo",
        description="ML Repo to log metrics and model to",
    ),
]

#### Resources

Allocate computing resources (CPU, memory, storage) for your service using the Resources option.<br>

- **CPU** refers to the computing power available to your application
- **Memory** refers to how much space your application has to hold and work with data while it's running
- **Ephemeral storage** is where your application can temporarily store files and data

Requests and Limits:

- **Request** is like asking for a certain amount of a resource. It's what your application initially asks for to start working properly.
- **Limit** is like setting a maximum value. It restricts how much of a resource (like CPU or memory) your application can use.

So for each category of resource you specify the Request and Limits


In [ ]:
from truefoundry.deploy import Resources

resources = Resources(
    memory_limit=500,
    memory_request=500,
    ephemeral_storage_limit=600,
    ephemeral_storage_request=600,
    cpu_limit=0.3,
    cpu_request=0.3,
)

### Step 2: Bring all of the configuration together via the Job Class and Deploy

To deploy your machine learning job, you need to create an instance of the `Job` class provided by the truefoundry library. This instance will encapsulate all the necessary configurations and parameters for deploying and managing your job.


In [ ]:
from truefoundry.deploy import Job

job = Job(name=name, image=image, resources=resources, params=params)

After configuring your deployment settings, you can deploy the job using the deploy method. Here we are replacing the WORKSPACE_FQN with the workspace_fqn we stored earlier.


In [ ]:
# Deploy the job
job.deploy(workspace_fqn=WORKSPACE_FQN, wait=False)

Once the build is complete, you will see a link to the dashboard after a message like `You can find the application on the dashboard:-`. <br>Click on the link to access the deployment dashboard.


# Effortless Hyperparameter Experimentation

Once your deployment is active, navigate to your specific job by clicking on it. This action will open a dedicated dashboard displaying various job details, including the **Run Job** button.

![](https://files.readme.io/cfff7cd-Screenshot_2023-08-23_at_1.48.02_PM.png)

Clicking this button will trigger a modal to appear:

![](https://files.readme.io/971a7fe-Screenshot_2023-08-23_at_1.51.38_PM.png)

Within this modal, you can effortlessly adjust hyperparameter values for rapid experimentation.

After configuring the modal, submit it using the Run Job button. This action will redirect you to the Job Runs tab. Within a few moments, your job status should switch to Finished.

Proceed by clicking on the logs button to access your job's results:

![](https://files.readme.io/1b79056-Screenshot_2023-08-28_at_7.17.03_AM.png)

Now closing, clicking the purple **churn-train-job** badge will grant you access to the Key Metrics, Hyperparameters, Logged Model, and Associated Artifacts from the run.

![](https://files.readme.io/0113700-Screenshot_2023-08-28_at_7.14.36_AM.png)

# Additional Capabilities of Jobs

Let's delve into the advanced functionalities that Jobs offer, extending beyond deployment strategies:

- **Continuous Integration/Continuous Deployment (CI/CD) via TrueFoundry:** Integrate Jobs with TrueFoundry for streamlined CI/CD pipelines, ensuring efficient code integration, testing, and deployment.
- **Cron Jobs:** Schedule Jobs to run at specified intervals using cron-like expressions, automating recurring tasks and processes.
- **Job Parametrization:** Configure Jobs with parameters, allowing you to customize execution by providing dynamic input values.
- **Programmatic Job Triggers:** Trigger Jobs programmatically via APIs, enabling seamless automation and integration with external systems.
- **Additional Configurations:** Access a range of supplementary configurations to fine-tune job behavior and optimize performance.
